In [ ]:
from gnucash_business_reports.builder import GnuCash_Data_Analysis
import numpy as np
from great_tables import GT, md
import polars as pl
import polars.selectors as cs
import pandas as pd
from pathlib import Path


gda = GnuCash_Data_Analysis()
gda.year = 2024

In [ ]:
gda.sanity_checker()

In [ ]:
gda.get_balance_sheet()

In [ ]:
# print(gda.get_executive_summary(include_depreciation=False))
# df = gda.get_farm_cash_transactions(include_depreciation=True)
# print(df.dtypes)
# print(df.sort_values("account_code").groupby(["account_code", "account_name"]).sum())
# print(gda.get_summary_by_account())

# flex = gda.flexible_lease_calculator()

# flex.to_csv("export/flex.csv")
# print(gda.get_corporation_value())
# print(gda.get_personal_business_expenses())

# gda.get_production().to_csv(f"export/{gda.year}-production.csv")

In [ ]:
gda.get_balance_sheet_details()

In [ ]:
grain = gda.get_grain_invoices()
open_contracts = grain[grain["Fulfilled"] == False].set_index("Invoice", drop=True)
print(open_contracts[["Crop", "Bushels", "Code"]])
open_contracts[["Crop", "Bushels", "Code"]].groupby(["Crop", "Code"]).sum()

In [ ]:
open_contracts[["Crop", "Bushels", "Code"]].groupby(["Crop", "Code"]).sum().to_html()

In [ ]:
# gda.get_all_cash_transactions().groupby(["account_desc"]).agg("post_date", np.max)
# latest_tx = gda.get_all_transactions().groupby(["account_desc"]).agg(max_date=("post_date", max))
latest_tx = gda.get_all_transactions().groupby(["account_desc"])["post_date"].max() #.reset_index().set_index("account_desc")
latest_tx

In [ ]:
latest = latest_tx.loc[latest_tx.index.str.match("Delivered")]
last_delivery = latest.max()

In [ ]:
gda.year = 2024
grain = (
            gda.get_commodity_stock_values(["account_name", "account_desc", "commodity_guid"])
        ).reset_index().set_index(["account_desc"])
grain["abs_qty"] = abs(grain["qty"])
grain = grain.join(latest_tx)
grain[["abs_qty", "post_date"]]

In [ ]:
df = pd.pivot_table(grain, values="abs_qty", index="account_name", columns="account_desc").fillna(0)
df["Contracted"] = df["Contracted Corn"] + df["Contracted Soybeans"]
df["Delivered"] = df["Delivered Corn"] + df["Delivered Soybeans"] + 3000
df["Harvested"] = df["Harvested Corn"] + df["Harvested Soybeans"] + 6000
df = df[["Contracted", "Delivered", "Harvested"]]
df["Total"] = df["Delivered"] + df["Harvested"]
df["pct"] = df["Delivered"] / df["Contracted"]
df

In [ ]:
def create_bar(prop_fill: float, max_width: int, height: int) -> str:
    """Create divs to represent prop_fill as a bar."""
    if prop_fill > 1:
        prop_fill = 1
    width = round(max_width * prop_fill, 2)
    px_width = f"{width}px"
    return f"""\
    <div style="width: {max_width}px; background-color: lightgrey;">\
        <div style="height:{height}px;width:{px_width};background-color:green;"></div>\
    </div>\
    """

df = pl.from_pandas(df.reset_index())

res = (
    df.with_columns(
        (pl.col("Delivered") / pl.col("Contracted")).alias("raw_perc"),
        (pl.col("account_name").str.to_lowercase() + ".png").alias("icon"),
    )
    .head(9)
    .with_columns(
        pl.col("raw_perc")
          .map_elements(lambda x: create_bar(x, max_width=75, height=20))
          .alias("Progress")
    )
    .select("icon", "Contracted", "Delivered", "Harvested", "Total", "Progress")
)
res

In [ ]:
table = (
    GT(res, rowname_col="icon")
    .tab_header(title=f"{gda.year} Harvest",
                subtitle="Progress towards filling contracts"
                )
    .tab_stubhead(label="Crop")
    # .tab_spanner("Earnings", cs.contains("Earnings"))
    .fmt_number(["Contracted", "Delivered", "Harvested", "Total"], decimals=0)
    # .tab_options(table_font_size="300%")
    
    # .cols_label(**{
    #     "Total Earnings": "Total $M",
    #     "Off-the-Field Earnings": "Off field $M",
    #     "Off-the-Field Earnings Perc": "Off field %"
    # })
    # .fmt_number(["Total Earnings", "Off-the-Field Earnings"], scale_by = 1/1_000_000, decimals=1)
    .fmt_image("icon", path="./img/")
    .tab_source_note(
        md(
            '<br><div style="text-align: center;">'
            "GNUCash Accounting"
            f" | Last Recorded Delivery: {last_delivery.strftime('%Y-%m-%d')}"
            "</div>"
            "<br>"
        )
    )
)
table

In [ ]:
html = table.as_raw_html()
path = Path("./table.html")
path.write_text(html)

In [ ]:
gda.year = 2023
production = GT(gda.get_production().reset_index())
gda.year = 2024
production